In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/isaacduong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/isaacduong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
BMBF = pd.read_csv('./BMBF.csv',encoding='latin1',sep=';')
BMBF

,"=""FKZ""","=""Ressort""","=""Referat""","=""PT""","=""Arb.-Einh.""","=""Zuwendungsempfänger""","=""Gemeindekennziffer""","=""Stadt/Gemeinde""","=""Ort""","=""Bundesland""",...,"=""Thema""","=""Leistungsplansystematik""","=""Klartext Leistungsplansystematik""","=""Laufzeit von""","=""Laufzeit bis""","=""Fördersumme in EUR""","=""Förderprofil""","=""Verbundprojekt""","=""Förderart""",Unnamed: 26
0,"=""03F0212C/7""","=""BMBF""","=""724""","=""PT-J""","=""MGS1""","=""Universität Rostock""","=""13003000""","=""Rostock""","=""Rostock""","=""Mecklenburg-Vorpommern""",...,"=""Verbundprojekt MESSIN: Entwicklung und Syste...","=""FB4010""","=""Technik-/Infrastrukturentwicklung und -berei...","=""01.01.1998""","=""31.08.2000""","648.462,00","=""Forschung und Entwicklung zur Daseinsvorsorge""","=""MESSIN""","=""PDIR""",NaN
1,"=""D251800 /0""","=""BMBF""","=""314""","=""BIBB""","=""A3.3""","=""Ministerium für Wirtschaft, Arbeit und Wohnu...","=""08111000""","=""Stuttgart""","=""Stuttgart""","=""Baden-Württemberg""",...,"=""Prozeßorientierte Entwicklungsplanung und Qu...","=""OB1050""","=""Versuchs- und Modelleinrichtungen und -progr...","=""01.07.1997""","=""30.06.2000""","96.634,00","=""Forschung und Entwicklung zur Daseinsvorsorge""","=""""","=""PDIR""",NaN
2,"=""D251800B/2""","=""BMBF""","=""314""","=""BIBB""","=""A3.3""","=""Ministerium für Wirtschaft, Arbeit und Wohnu...","=""08111000""","=""Stuttgart""","=""Stuttgart""","=""Baden-Württemberg""",...,"=""Wiss. Begl. zum MV 'Prozeßorientierte Entwic...","=""OB1050""","=""Versuchs- und Modelleinrichtungen und -progr...","=""01.07.1997""","=""30.06.2000""","157.457,00","=""Forschung und Entwicklung zur Daseinsvorsorge""","=""""","=""PDIR""",NaN
3,"=""50WM9821/6""","=""BMWi""","=""IVD5""","=""DLR-RFM""","=""AR-AF""","=""Fraunhofer-Gesellschaft zur Förderung der an...","=""09162000""","=""München""","=""München""","=""Bayern""",...,"=""Untersuchung des Schäumens von Metallen unte...","=""IB3010""","=""Forschung unter Weltraumbedingungen - Materi...","=""01.03.1998""","=""31.12.2001""","452.356,00","=""Technologie- und Innovationsförderung""","=""""","=""PDIR""",NaN
4,"=""50WB9716/8""","=""BMWi""","=""IVD5""","=""DLR-RFM""","=""AR-AF""","=""Universität Hohenheim""","=""08111000""","=""Stuttgart""","=""Stuttgart""","=""Baden-Württemberg""",...,"=""Der Einfluß verschiedener Gravitationsbeding...","=""IB3020""","=""Forschung unter Weltraumbedingungen - Biowis...","=""01.01.1998""","=""31.03.2002""","358.551,00","=""Technologie- und Innovationsförderung""","=""""","=""PDIR""",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272239,"=""03EN4022J""","=""BMWi""","=""IIC6""","=""PT-J""","=""ESN4""","=""Georgsmarienhütte GmbH""","=""03459019""","=""Georgsmarienhütte""","=""Georgsmarienhütte""","=""Niedersachsen""",...,"=""Verbundvorhaben: SiPro - Simulationsbasierte...","=""EA3251""","=""Energiesparende Industrieverfahren - Fertigu...","=""01.01.2022""","=""31.12.2024""","227.401,00","=""Technologie- und Innovationsförderung""","=""SiPro""","=""PDIR""",NaN
272240,"=""67K18622""","=""BMU""","=""IKIII3""","=""PT-ZUG""","=""NKI""","=""Amt Bokhorst-Wankendorf""","=""01057085""","=""Wankendorf""","=""Wankendorf""","=""Schleswig-Holstein""",...,"=""KSI: Sanierung der Innenbeleuchtung im Dorfg...","=""FA1913""","=""KSI - Klimaschutztechnologien bei der Stromn...","=""01.03.2022""","=""28.02.2023""","10.802,00","=""Forschung und Entwicklung zur Daseinsvorsorge""","=""""","=""PDIR""",NaN
272241,"=""13N15865""","=""BMBF""","=""514""","=""PT-VDI""","=""TZ-PT""","=""kiutra GmbH""","=""09162000""","=""München""","=""München""","=""Bayern""",...,"=""Verbundprojekt: Materials in Quantum Computi...","=""L01010""","=""Quantentechnologien: Quantencomputing und -s...","=""01.09.2021""","=""31.05.2024""","228.580,00","=""Technologie- und Innovationsförderung""","=""Materials in Quantum Computing""","=""PDIR""",NaN
272242,"=""13N15863""","=""BMBF""","=""514""","=""PT-VDI""","=

In [3]:
def cleaned_data(dataframe=BMBF):
    # remove invalid token in columns name
    columns = [c.replace('\"','').replace('=','') for c in BMBF.columns]
    BMBF.columns = columns
    BMBF.drop('Unnamed: 26',axis = 1, inplace = True)
    # remove invalid token in columns values
    for col in columns[:-1]:
        BMBF[col] = BMBF[col].apply(lambda v: v.replace('\"','').replace('=',''))
    return BMBF

In [4]:
BMBF = cleaned_data()
BMBF


,FKZ,Ressort,Referat,PT,Arb.-Einh.,Zuwendungsempfänger,Gemeindekennziffer,Stadt/Gemeinde,Ort,Bundesland,...,Staat.1,Thema,Leistungsplansystematik,Klartext Leistungsplansystematik,Laufzeit von,Laufzeit bis,Fördersumme in EUR,Förderprofil,Verbundprojekt,Förderart
0,03F0212C/7,BMBF,724,PT-J,MGS1,Universität Rostock,13003000,Rostock,Rostock,Mecklenburg-Vorpommern,...,Deutschland,Verbundprojekt MESSIN: Entwicklung und Systemi...,FB4010,Technik-/Infrastrukturentwicklung und -bereits...,01.01.1998,31.08.2000,"648.462,00",Forschung und Entwicklung zur Daseinsvorsorge,MESSIN,PDIR
1,D251800 /0,BMBF,314,BIBB,A3.3,"Ministerium für Wirtschaft, Arbeit und Wohnung...",08111000,Stuttgart,Stuttgart,Baden-Württemberg,...,Deutschland,Prozeßorientierte Entwicklungsplanung und Qual...,OB1050,Versuchs- und Modelleinrichtungen und -program...,01.07.1997,30.06.2000,"96.634,00",Forschung und Entwicklung zur Daseinsvorsorge,,PDIR
2,D251800B/2,BMBF,314,BIBB,A3.3,"Ministerium für Wirtschaft, Arbeit und Wohnung...",08111000,Stuttgart,Stuttgart,Baden-Württemberg,...,Deutschland,Wiss. Begl. zum MV 'Prozeßorientierte Entwickl...,OB1050,Versuchs- und Modelleinrichtungen und -program...,01.07.1997,30.06.2000,"157.457,00",Forschung und Entwicklung zur Daseinsvorsorge,,PDIR
3,50WM9821/6,BMWi,IVD5,DLR-RFM,AR-AF,Fraunhofer-Gesellschaft zur Förderung der ange...,09162000,München,München,Bayern,...,Deutschland,Untersuchung des Schäumens von Metallen unter ...,IB3010,Forschung unter Weltraumbedingungen - Material...,01.03.1998,31.12.2001,"452.356,00",Technologie- und Innovationsförderung,,PDIR
4,50WB9716/8,BMWi,IVD5,DLR-RFM,AR-AF,Universität Hohenheim,08111000,Stuttgart,Stuttgart,Baden-Württemberg,...,Deutschland,Der Einfluß verschiedener Gravitationsbedingun...,IB3020,Forschung unter Weltraumbedingungen - Biowisse...,01.01.1998,31.03.2002,"358.551,00",Technologie- und Innovationsförderung,,PDIR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272239,03EN4022J,BMWi,IIC6,PT-J,ESN4,Georgsmarienhütte GmbH,03459019,Georgsmarienhütte,Georgsmarienhütte,Niedersachsen,...,Deutschland,Verbundvorhaben: SiPro - Simulationsbasiertes ...,EA3251,Energiesparende Industrieverfahren - Fertigung...,01.01.2022,31.12.2024,"227.401,00",Technologie- und Innovationsförderung,SiPro,PDIR
272240,67K18622,BMU,IKIII3,PT-ZUG,NKI,Amt Bokhorst-Wankendorf,01057085,Wankendorf,Wankendorf,Schleswig-Holstein,...,Deutschland,KSI: Sanierung der Innenbeleuchtung im Dorfgem...,FA1913,KSI - Klimaschutztechnologien bei der Stromnut...,01.03.2022,28.02.2023,"10.802,00",Forschung und Entwicklung zur Daseinsvorsorge,,PDIR
272241,13N15865,BMBF,514,PT-VDI,TZ-PT,kiutra GmbH,09162000,München,München,Bayern,...,Deutschland,Verbundprojekt: Materials in Quantum Computing...,L01010,Quantentechnologien: Quantencomputing und -sim...,01.09.2021,31.05.2024,"228.580,00",Technologie- und Innovationsförderung,Materials in Quantum Computing,PDIR
272242,13N15863,BMBF,514,PT-VDI,TZ-PT,Keysight Technologies Deutschland GmbH,08115003,Böblingen,Böblingen,Baden-Württemberg,...,Deutschland,Verbundprojekt: Materials in Quantum Computing...,L01010,Quantentechnologien: Quantencomputing und -sim...,01.09.2021,31.05.2024,"99.593,00",Technologie- und Innovationsförderung,Materials in Quantum Computing,PDIR


In [5]:
stop_words = set(stopwords.words('german'))
stemmer = SnowballStemmer("german")
def tokenize_and_stem(text):

    tokens = [word for word in nltk.word_tokenize(text)if word not in stop_words]
    stems = [stemmer.stem(t) for t in tokens]
    return stems


select n_features for best score,
max_features=[100,200,500,1000,2000,5000];
max_feature=1000 has highest silhouette score


In [6]:
tfidf_vectorizer = TfidfVectorizer(max_features=1000,
                                   use_idf=True, 
                                   tokenizer=tokenize_and_stem,
                                   ngram_range=(1,1))

X = tfidf_vectorizer.fit_transform(BMBF['Thema'])


from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=100, random_state=42)
X_new = svd.fit_transform(X)


In [115]:
#clustering using DBSCAN
%time bdscan = DBSCAN().fit(X)

CPU times: user 11min 35s, sys: 4min 22s, total: 15min 58s
Wall time: 24min 55s


In [116]:
#calculate score of clustering, silhouette score: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html
%time bdscan_silhouette_s= silhouette_score(X,bdscan.labels_)
print(f'silhouette_score using euclidean distance as metric: {bdscan_silhouette_s}')
print(f'n_clusters: {len(np.unique(bdscan.labels_))}')
pd.DataFrame({'labels':bdscan.labels_}).value_counts()

CPU times: user 16min 35s, sys: 4min 35s, total: 21min 11s
Wall time: 21min 20s
silhouette_score using euclidean distance as metric: 0.009934058263912625
n_clusters: 4602


labels
-1        150447
 3         20458
 15        10162
 3259       4127
 1367       4057
           ...  
 3962          3
 3945          3
 4593          3
 2616          3
 4381          2
Length: 4602, dtype: int64

In [118]:
BMBF['group_DBSCAN']= bdscan.labels_

**gridsearch for n_clusters with best silhouette score
n_clusters= 5000,10000,20000,40000,60000,80000
for n_cluster in n_clusters:

In [85]:
#clustering using Kmean
mbkmean = MiniBatchKMeans(n_clusters= 70000,verbose=2, random_state=42)


In [86]:
%time mbkmean.fit(X)

Init 1/3 with method: k-means++
Inertia for init 1/3: 17934.351190
Init 2/3 with method: k-means++
Inertia for init 2/3: 17779.337447
Init 3/3 with method: k-means++
Inertia for init 3/3: 17860.838493
Minibatch iteration 1/272300: mean batch inertia: 0.129372, ewa inertia: 0.129372 
Minibatch iteration 2/272300: mean batch inertia: 0.130864, ewa inertia: 0.129373 
Minibatch iteration 3/272300: mean batch inertia: 0.157558, ewa inertia: 0.129393 
Minibatch iteration 4/272300: mean batch inertia: 0.121613, ewa inertia: 0.129388 
Minibatch iteration 5/272300: mean batch inertia: 0.145309, ewa inertia: 0.129399 
Minibatch iteration 6/272300: mean batch inertia: 0.123104, ewa inertia: 0.129395 
Minibatch iteration 7/272300: mean batch inertia: 0.111318, ewa inertia: 0.129381 
Minibatch iteration 8/272300: mean batch inertia: 0.166773, ewa inertia: 0.129409 
Minibatch iteration 9/272300: mean batch inertia: 0.104319, ewa inertia: 0.129390 
[MiniBatchKMeans] Reassigning 50 cluster centers.
Mi

MiniBatchKMeans(n_clusters=70000, random_state=42, verbose=2)

In [87]:
#silhouette score using metric euclidean distance n_clusters=70000
%time mbkmean_silhouette_s= silhouette_score(X,mbkmean.labels_)
mbkmean_silhouette_s

CPU times: user 21min 50s, sys: 6min 51s, total: 28min 42s
Wall time: 28min 52s


0.3813701534148748

In [114]:
pd.DataFrame({'labels':mbkmean.labels_}).value_counts()

n_clusters: 69984


labels
4         10344
3          4127
22         4062
15         1219
50         1168
          ...  
45411         1
21982         1
45409         1
45408         1
50750         1
Length: 69984, dtype: int64

In [89]:
BMBF['group_mbkmean']=mbkmean.labels_

,Thema,group_mbkmean
0,Verbundprojekt MESSIN: Entwicklung und Systemi...,63120
1,Prozeßorientierte Entwicklungsplanung und Qual...,68182
2,Wiss. Begl. zum MV 'Prozeßorientierte Entwickl...,29631
3,Untersuchung des Schäumens von Metallen unter ...,15707
4,Der Einfluß verschiedener Gravitationsbedingun...,38579
...,...,...
272239,Verbundvorhaben: SiPro - Simulationsbasiertes ...,22518
272240,KSI: Sanierung der Innenbeleuchtung im Dorfgem...,17
272241,Verbundprojekt: Materials in Quantum Computing...,3408
272242,Verbundprojekt: Materials in Quantum Computing...,37506


In [119]:
BMBF

,FKZ,Ressort,Referat,PT,Arb.-Einh.,Zuwendungsempfänger,Gemeindekennziffer,Stadt/Gemeinde,Ort,Bundesland,...,Leistungsplansystematik,Klartext Leistungsplansystematik,Laufzeit von,Laufzeit bis,Fördersumme in EUR,Förderprofil,Verbundprojekt,Förderart,group_mbkmean,group_DBSCAN
0,03F0212C/7,BMBF,724,PT-J,MGS1,Universität Rostock,13003000,Rostock,Rostock,Mecklenburg-Vorpommern,...,FB4010,Technik-/Infrastrukturentwicklung und -bereits...,01.01.1998,31.08.2000,"648.462,00",Forschung und Entwicklung zur Daseinsvorsorge,MESSIN,PDIR,63120,-1
1,D251800 /0,BMBF,314,BIBB,A3.3,"Ministerium für Wirtschaft, Arbeit und Wohnung...",08111000,Stuttgart,Stuttgart,Baden-Württemberg,...,OB1050,Versuchs- und Modelleinrichtungen und -program...,01.07.1997,30.06.2000,"96.634,00",Forschung und Entwicklung zur Daseinsvorsorge,,PDIR,68182,-1
2,D251800B/2,BMBF,314,BIBB,A3.3,"Ministerium für Wirtschaft, Arbeit und Wohnung...",08111000,Stuttgart,Stuttgart,Baden-Württemberg,...,OB1050,Versuchs- und Modelleinrichtungen und -program...,01.07.1997,30.06.2000,"157.457,00",Forschung und Entwicklung zur Daseinsvorsorge,,PDIR,29631,-1
3,50WM9821/6,BMWi,IVD5,DLR-RFM,AR-AF,Fraunhofer-Gesellschaft zur Förderung der ange...,09162000,München,München,Bayern,...,IB3010,Forschung unter Weltraumbedingungen - Material...,01.03.1998,31.12.2001,"452.356,00",Technologie- und Innovationsförderung,,PDIR,15707,0
4,50WB9716/8,BMWi,IVD5,DLR-RFM,AR-AF,Universität Hohenheim,08111000,Stuttgart,Stuttgart,Baden-Württemberg,...,IB3020,Forschung unter Weltraumbedingungen - Biowisse...,01.01.1998,31.03.2002,"358.551,00",Technologie- und Innovationsförderung,,PDIR,38579,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272239,03EN4022J,BMWi,IIC6,PT-J,ESN4,Georgsmarienhütte GmbH,03459019,Georgsmarienhütte,Georgsmarienhütte,Niedersachsen,...,EA3251,Energiesparende Industrieverfahren - Fertigung...,01.01.2022,31.12.2024,"227.401,00",Technologie- und Innovationsförderung,SiPro,PDIR,22518,4600
272240,67K18622,BMU,IKIII3,PT-ZUG,NKI,Amt Bokhorst-Wankendorf,01057085,Wankendorf,Wankendorf,Schleswig-Holstein,...,FA1913,KSI - Klimaschutztechnologien bei der Stromnut...,01.03.2022,28.02.2023,"10.802,00",Forschung und Entwicklung zur Daseinsvorsorge,,PDIR,17,2387
272241,13N15865,BMBF,514,PT-VDI,TZ-PT,kiutra GmbH,09162000,München,München,Bayern,...,L01010,Quantentechnologien: Quantencomputing und -sim...,01.09.2021,31.05.2024,"228.580,00",Technologie- und Innovationsförderung,Materials in Quantum Computing,PDIR,3408,-1
272242,13N15863,BMBF,514,PT-VDI,TZ-PT,Keysight Technologies Deutschland GmbH,08115003,Böblingen,Böblingen,Baden-Württemberg,...,L01010,Quantentechnologien: Quantencomputing und -sim...,01.09.2021,31.05.2024,"99.593,00",Technologie- und Innovationsförderung,Materials in Quantum Computing,PDIR,37506,-1


In [120]:
BMBF.to_csv('clusteredBMBF.csv',index=False)